# Gestión y uso de metadatos: Dublin Core y XML


## Objetivos de la sesión
- Comprender la motivación y los componentes principales del esquema Dublin Core.
- Recordar la estructura básica de un documento XML y buenas prácticas para crearlo.
- Practicar el parseo y la consulta de documentos XML con `xml.etree.ElementTree`.
- Preparar a los alumnos para resolver de forma autónoma ejercicios prácticos apoyándose en herramientas como ChatGPT de manera crítica.


### Parte guiada (en clase)
En esta primera parte trabajaremos juntos. Puedes ejecutar las celdas conforme avancemos en la explicación. Si durante la sesión utilizas herramientas de IA (como ChatGPT), contrasta siempre sus respuestas con la documentación oficial y con tus notas.


#### Librerías necesarias
Usaremos la librería estándar `xml.etree.ElementTree` para construir y analizar documentos XML. También emplearemos `requests` para descargar ejemplos reales de metadatos publicados en la web.


In [ ]:
import xml.etree.ElementTree as ET
import requests
from xml.dom import minidom


#### Describir un recurso con Dublin Core
Empecemos con un ejemplo sencillo: el cuadro **El grito**, de Edvard Munch. Pensaremos qué elementos del esquema Dublin Core nos sirven para describirlo.


In [ ]:
painting_dc = {
    "title": "The Scream",
    "creator": "Edvard Munch",
    "subject": "Expressionism, Painting",
    "description": "Iconic expressionist artwork depicting a figure holding their face against a dramatic sky.",
    "publisher": "National Museum of Norway",
    "contributor": "",
    "date": "1893",
    "type": "StillImage",
    "format": "Oil, tempera and pastel on cardboard",
    "identifier": "http://hdl.handle.net/12345/scream",
    "source": "National Museum Collection",
    "language": "",
    "relation": "",
    "coverage": "Oslo, Norway",
    "rights": "© Public domain"
}
painting_dc


A partir de este diccionario generaremos un documento XML sencillo respetando la sintaxis básica: etiqueta de apertura, contenido y etiqueta de cierre. Dublin Core recomienda declarar los elementos en el espacio de nombres `http://purl.org/dc/elements/1.1/`.


In [ ]:
def build_dc_xml(dc_dict):
    dc_namespace = "http://purl.org/dc/elements/1.1/"
    root = ET.Element("metadata", {"xmlns:dc": dc_namespace})

    for key, value in dc_dict.items():
        elem = ET.SubElement(root, f"dc:{key}")
        elem.text = value

    return root


dc_tree = build_dc_xml(painting_dc)
raw_xml = ET.tostring(dc_tree, encoding="utf-8")
raw_xml


In [ ]:
def pretty_print(xml_bytes):
    parsed = minidom.parseString(xml_bytes)
    return parsed.toprettyxml(indent="  ")


print(pretty_print(raw_xml))


Podemos guardar este XML en disco para reutilizarlo o compartirlo con otros sistemas que sepan leer Dublin Core.


In [ ]:
with open("el_grito_dc.xml", "wb") as f:
    f.write(raw_xml)


print("Documento guardado en el directorio actual.")


#### Parsear el XML generado
Veamos ahora cómo leer el documento que acabamos de crear y acceder a sus elementos.


In [ ]:
tree = ET.parse("el_grito_dc.xml")
root = tree.getroot()


for child in root:
    print(f"Etiqueta: {child.tag} | Valor: {child.text}")


#### Búsquedas específicas y namespaces
Cuando trabajamos con espacios de nombres necesitamos proporcionar un diccionario de prefijos al realizar consultas XPath simplificadas con `ElementTree`.


In [ ]:
namespaces = {"dc": "http://purl.org/dc/elements/1.1/"}


# Buscar el título y el creador usando findall con prefijos
for field in ["title", "creator"]:
    results = root.findall(f"dc:{field}", namespaces)
    for elem in results:
        print(f"{field.title()}: {elem.text}")


#### Trabajar con un XML externo real
En muchos repositorios científicos los metadatos se publican en XML. A continuación descargaremos un ejemplo de registro Dublin Core expuesto por Europeana. Si no tienes conexión a internet, pide al profesorado el archivo descargado previamente (`europeana_dc.xml`).


In [ ]:
url = "https://www.europeana.eu/api/v2/record/92037/_http___www_bl_uk_onlinegallery_onlineex_treasures_evanthomas_uk_full_0037597464_xml.json?wskey=apidemo"
response = requests.get(url)


if response.ok:
    data = response.json()
    xml_snippet = data["object"]["proxies"][0]["dcDescription"][0]
    print("Descripción recuperada:
", xml_snippet)
else:
    print("No fue posible descargar el ejemplo. Comprueba tu conexión o usa el archivo local.")


El fragmento descargado puede convertirse en XML reutilizando la función `build_dc_xml` o analizándose directamente si ya está serializado. Aquí mostramos cómo parsear un archivo externo (`datacite_example.xml`) que debe estar disponible en la carpeta de trabajo antes de ejecutar la celda.


In [ ]:
try:
    datacite_tree = ET.parse("datacite_example.xml")
    datacite_root = datacite_tree.getroot()
    print("Raíz:", datacite_root.tag)
    print("Elementos dc:title encontrados:")
    for elem in datacite_root.findall('.//{http://purl.org/dc/elements/1.1/}title'):
        print("  -", elem.text)
except FileNotFoundError:
    print("Descarga o copia previamente 'datacite_example.xml' para poder analizarlo en clase.")


---

### Ejercicios individuales
Trabaja ahora por tu cuenta. Puedes colaborar con tus compañeros y, si recurres a herramientas de IA, explica cómo las usaste y valida las respuestas contrastándolas con el XML real.


#### Ejercicio 1
Descarga los metadatos en formato **DataCite 4** de un objeto digital (por ejemplo, desde Zenodo con la opción "Export to DataCite").
Recorre todos los términos presentes en el XML y clasifícalos en una tabla según sean metadatos **descriptivos**, **administrativos** o **estructurales**.
Justifica brevemente cualquier decisión dudosa o ambigua.


In [ ]:
# Escribe aquí tu solución para el Ejercicio 1.
# Pista: puedes construir una lista de tuplas (etiqueta, categoría) e incluso apoyarte en pandas para generar la tabla.


#### Ejercicio 2
A partir del mismo XML de DataCite, genera una salida tabular que incluya para cada elemento: **etiqueta**, **contenido textual** (si lo hay) y **atributos**.
Si un elemento aparece varias veces, incluye todas las ocurrencias.


In [ ]:
# Solución del Ejercicio 2.
# Pista: itera sobre los elementos con `iter()` o `iterfind()` y construye un DataFrame o imprime filas formateadas.


#### Ejercicio 3
Localiza un ejemplo de metadatos en XML de un estándar disciplinar (consulta el directorio de esquemas de la RDA si necesitas ideas).
Identifica qué elementos del estándar pueden mapearse a los términos de Dublin Core y documenta las correspondencias (por ejemplo, en una tabla o diccionario).
Señala también los elementos sin equivalencia clara.


In [ ]:
# Solución del Ejercicio 3.
# Pista: crea un diccionario {"Elemento estándar": "Elemento Dublin Core"} y comenta los casos sin mapeo directo.


#### Ejercicio 4
Analiza el esquema trabajado (DataCite o el estándar disciplinar) para identificar qué términos apoyan cada característica de los principios **FAIR**: Findable, Accessible, Interoperable y Reusable.
Explica brevemente por qué cada término contribuye al principio correspondiente e indica posibles carencias.


In [ ]:
# Solución del Ejercicio 4.
# Pista: organiza la respuesta con un diccionario o tabla que relacione cada principio FAIR con los elementos del esquema.


#### Buenas prácticas con herramientas de IA
- Documenta cuándo y para qué utilizas ChatGPT u otras IA durante la resolución de los ejercicios.
- Comprueba siempre el resultado con tus propias pruebas.
- Si detectas inconsistencias, anótalas y compártelas en clase para aprender en conjunto.
